In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
import json
from flask import jsonify
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from dash.exceptions import PreventUpdate



# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PythonCodeMod5Final import AnimalShelter

# Data Manipulation / Model
# FIXEDME: Update with your username, password, and CRUD Python module name
username = "aacuser"
password = "best_password_ever"

# Connect to the database via the CRUD Module
shelter = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32197, 'AAC', 'animals')

# class read method must support return of a list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column, and that is going to have an
# invalid object type of 'ObjectID', which will cause the data_table to crash - so we remove
# it in the DataFrame here.
df.drop(columns=['_id'], inplace=True)

# Dashboard Layout / View
app = JupyterDash(__name__)

# FIXEDME: Replace '.png' with your own image filename
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard - Brianna De La Riva'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'wr'},
            {'label': 'Mountain Rescue', 'value': 'mr'},
            {'label': 'Disaster Rescue', 'value': 'dr'},
            {'label': 'Reset', 'value': 'resets'}
        ],
        value='wr',  # Default selected value
        labelStyle={'display': 'block'}  # Align radio items vertically
    ),


    # FIXEDME: Add in code for the interactive filtering options. For example, Radio buttons, drop-down, checkboxes, etc.
    # buttons added above, water rescue, mountain rescue, disaster rescue, and reset
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         #data=df.to_dict('records'),
                         data=[],
                         editable=False, filter_action="native", sort_action="native",
                         sort_mode="multi",
                         column_selectable=False, row_selectable="single", row_deletable=False, selected_rows=[0],
                         page_action="native", page_current=0, page_size=10),
    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])


# Interaction Between Components / Controller
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')]
)
def update_data(filter_type):
    print(f'filter_type: {filter_type}')
    
    # Call the update_dashboard function and return the filtered data
    filtered_data = update_dashboard(filter_type)
    if filtered_data is not None:
        if isinstance(filtered_data, pd.DataFrame):
            print(f'records')
            return filtered_data.to_dict('records')
        else:
            #print is for debugging purposes
            print(f'none1')
            # Handle the case when filtered_data is a list or other non-DataFrame type
            return []
    else:
        #print is for debugging purposes
        print(f'none2')
        return []



def update_dashboard(filter_type):
    query = {}

    if filter_type == 'wr':
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'mr':
        query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                              'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_type == 'dr':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever',
                              'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    elif filter_type == 'resets':
        # No specific filter, show all data
        query = {}

    print(f'query: {query}')

    filtered_df = pd.DataFrame.from_records(shelter.read(query))
    filtered_df.drop(columns=['_id'], inplace=True)



    return filtered_df





# Display the breeds of animals based on the quantity represented in the data table
# This callback will update the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)
def update_graphs(viewData, filter_type):
    if viewData is None or not viewData:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    # Filter the data based on the selected filter type
    filtered_df = update_dashboard(filter_type)

    # Group the filtered data by the 'breed' column and calculate the count for each breed
    breed_counts = filtered_df['breed'].value_counts()

    # Create a pie chart using Plotly Express
    pie_chart = px.pie(names=breed_counts.index, values=breed_counts.values, title=f'Breed Distribution - {filter_type}')

    # Display the pie chart
    return dcc.Graph(figure=pie_chart)




# This callback will highlight a cell on the data table when the user selects it

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []

    styles = [{
        'if': {'column_id': col},
        'background_color': '#D2F3FF'
    } for col in selected_columns]

    return styles




# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or not viewData:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty or len(dff.columns) < 15:  
        return []

    row = index[0] if index else 0

    lat_column_index = 13
    lon_column_index = 14
    animal_name_column_index = 9
    tooltip_column_index = 4

    # Check if the specified columns exist in the DataFrame
    if lat_column_index >= len(dff.columns) or lon_column_index >= len(dff.columns) \
            or animal_name_column_index >= len(dff.columns) or tooltip_column_index >= len(dff.columns):
        return []

    lat = dff.iloc[row, lat_column_index]
    lon = dff.iloc[row, lon_column_index]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, tooltip_column_index]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, animal_name_column_index])
                ])
            ])
        ])
    ]


app.run_server(debug=True)



Dash app running on http://127.0.0.1:18814/
filter_type: wr
query: {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome': 'Intact Female', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
records
query: {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome': 'Intact Female', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
filter_type: wr
query: {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome': 'Intact Female', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
records
query: {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome': 'Intact Female', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
query: {'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 'sex_upon_outcome': 'Intact 